In [ ]:
pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 12.1 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, Namespace

def load_ontology(ontology_path):
    g = Graph()
    g.parse(ontology_path, format="ttl")
    GO = Namespace("http://example.org/go#")
    g.bind("go", GO)
    return g, GO

ontology_path = "first.ttl"
graph, GO = load_ontology(ontology_path)
print(f"Онтология загружена: {len(graph)} триплетов")


Онтология загружена: 91 триплетов


In [ ]:
import os
import json
import pandas as pd
import re
from rdflib import Graph, Namespace, RDF, RDFS, OWL, URIRef, Literal, XSD

def clean_uri_component(text):
    return re.sub(r'[^\w\-]', '_', text.strip())

def init_graph():
    g = Graph()
    GO = Namespace("http://example.org/go#")
    g.bind("go", GO)
    g.bind("owl", OWL)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("xsd", XSD)
    for prefix, ns in graph.namespaces():
        g.bind(prefix, ns)
    return g, GO

def parse_textgrid(textgrid_path):
    with open(textgrid_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    tiers = {}
    current_tier = None
    active = False
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith("name ="):
            current_tier = line.split("=")[1].strip().strip('"')
            tiers[current_tier] = []
            active = True
        if active and "intervals [" in line:
            xmin = float(lines[i + 1].split("=")[1].strip())
            xmax = float(lines[i + 2].split("=")[1].strip())
            text = lines[i + 3].split("=")[1].strip().strip('"')
            tiers[current_tier].append((xmin, xmax, text))
    return tiers

def process_game(json_path, base_dir, g, GO):
    with open(json_path, "r", encoding="utf-8") as f:
        game_data = json.load(f)

    game_id = game_data["meta"]["game_id"]
    game_uri = URIRef(f"http://example.org/go#{game_id}")
    g.add((game_uri, RDF.type, GO.Game))

    goterm_index = {}

    for idx, move_info in enumerate(game_data["moves_info_link"]):
        move_uri = URIRef(f"http://example.org/go#Move_{game_id}_{idx}")
        g.add((move_uri, RDF.type, GO.Move))
        g.add((move_uri, GO.moves, Literal(move_info["moves"])))
        g.add((game_uri, GO.hasMove, move_uri))

        eval_val = move_info.get("evaluation", "")
        if eval_val and ":" in eval_val:
            try:
                winner, winrate = eval_val.split(":")
                winner = winner.strip()
                winrate = winrate.strip()
                eval_uri = URIRef(f"http://example.org/go#Eval_{game_id}_{idx}")
                g.add((eval_uri, RDF.type, GO.Evaluation))
                g.add((eval_uri, GO.winner, Literal(winner, datatype=XSD.string)))
                g.add((eval_uri, GO.winRate, Literal(float(winrate), datatype=XSD.float)))
                g.add((move_uri, GO.hasEvaluation, eval_uri))
            except Exception as e:
                print(f"[!] Ошибка разбора оценки '{eval_val}': {e}")

        tsv_path = os.path.join(base_dir, move_info["tsv"])
        if os.path.exists(tsv_path):
            df = pd.read_csv(tsv_path, sep="\t", comment="#", header=None, quoting=3, keep_default_na=False)
            if df.shape[1] >= 11:
                df.columns = [
                    "token_id", "start", "text", "GoTerm", "LexicalUsage", "best_move", "score", "win_rate",
                    "PartOfSpeech", "concept_type", "stage", "empty"
                ]
                for i, row in df.iterrows():
                    term_key = row["GoTerm"].strip()
                    if term_key and term_key != "_":
                        cleaned_key = clean_uri_component(term_key)
                        if term_key not in goterm_index:
                            term_uri = URIRef(f"http://example.org/go#GoTerm_{game_id}_{cleaned_key}")
                            g.add((term_uri, RDF.type, GO.GoTerm))
                            g.add((term_uri, GO.termType, Literal(term_key)))
                            g.add((term_uri, GO.source, Literal("text")))
                            goterm_index[term_key] = term_uri
                        else:
                            term_uri = goterm_index[term_key]
                            prev_source = g.value(term_uri, GO.source)
                            if prev_source and "text" not in str(prev_source):
                                g.set((term_uri, GO.source, Literal("text,audio")))

                        g.add((move_uri, GO.hasTerm, term_uri))

                        concept_key = row["concept_type"].strip()
                        if concept_key and concept_key != "_":
                            concept_uri = URIRef(f"http://example.org/go#Concept_{game_id}_{idx}_{i}")
                            g.add((concept_uri, RDF.type, GO.StrategyConcept))
                            g.add((concept_uri, GO.conceptType, Literal(concept_key)))
                            g.add((term_uri, GO.relatedToStrategy, concept_uri))

                        stage_key = row["stage"].strip()
                        if stage_key and stage_key != "_":
                            stage_uri = URIRef(f"http://example.org/go#Stage_{game_id}_{idx}_{i}")
                            g.add((stage_uri, RDF.type, GO.GameStage))
                            g.add((stage_uri, GO.stage, Literal(stage_key)))
                            g.add((term_uri, GO.occursInStage, stage_uri))

        textgrid_path = os.path.join(base_dir, move_info["textgrid"]["textgrid_file_path"])
        if os.path.exists(textgrid_path):
            tiers = parse_textgrid(textgrid_path)

            for j, (xmin, xmax, text) in enumerate(tiers.get("speech_errors", [])):
                if text.strip() and text.strip() != "_":
                    key = text.strip()
                    err_uri = URIRef(f"http://example.org/go#SpeechError_{game_id}_{idx}_{j}")
                    g.add((err_uri, RDF.type, GO.SpeechError))
                    g.add((err_uri, GO.error_type, Literal(key)))
                    g.add((err_uri, GO.timestamp, Literal(f"{xmin:.2f}-{xmax:.2f}")))

                    g.add((move_uri, GO.hasError, err_uri))

            for j, (xmin, xmax, text) in enumerate(tiers.get("GoTerm", [])):
                if text.strip() and text.strip() != "_":
                    key = text.strip()
                    cleaned_key = clean_uri_component(key)
                    if key not in goterm_index:
                        term_uri = URIRef(f"http://example.org/go#GoTerm_{game_id}_{cleaned_key}")
                        g.add((term_uri, RDF.type, GO.GoTerm))
                        g.add((term_uri, GO.termType, Literal(key)))
                        g.add((term_uri, GO.source, Literal("audio")))
                        g.add((term_uri, GO.timestamp, Literal(f"{xmin:.2f}-{xmax:.2f}")))
                        goterm_index[key] = term_uri
                    else:
                        term_uri = goterm_index[key]
                        prev_source = g.value(term_uri, GO.source)
                        new_source = "audio"
                        if prev_source:
                            new_source = "text,audio" if "text" in str(prev_source) else "audio"
                        g.set((term_uri, GO.source, Literal(new_source)))
                        g.set((term_uri, GO.timestamp, Literal(f"{xmin:.2f}-{xmax:.2f}")))

                    g.add((move_uri, GO.hasTerm, term_uri))

    return g

base_dir = "/content"
json_file = os.path.join(base_dir, "game1.json")

graph, GO = init_graph()
graph = process_game(json_file, base_dir, graph, GO)

graph.serialize(destination=os.path.join(base_dir, "go_graph.ttl"), format="turtle",base="http://example.org/go#")
print(f"Создано триплетов: {len(graph)}")


Создано триплетов: 56


In [ ]:
from rdflib import Graph, RDF, Namespace

def clean_uris(input_file, output_file):
    g = Graph()
    g.parse(input_file, format="turtle")

    clean_g = Graph()

    for prefix, namespace in g.namespaces():
        clean_g.bind(prefix, namespace)

    clean_g.bind("rdf", RDF)

    for s, p, o in g:
        clean_g.add((s, p, o))

    has_rdf_predicate = any(p == RDF.type for s, p, o in clean_g)
    print("Есть ли в графе RDF.type?", has_rdf_predicate)

    ttl_data = clean_g.serialize(format="turtle", encoding="utf-8").decode("utf-8")

    prefix_line = "@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n"
    if prefix_line not in ttl_data:
        ttl_data = prefix_line + ttl_data

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(ttl_data)

    print(f"Очищенный граф сохранен в {output_file}")

if __name__ == "__main__":
    clean_uris("go_graph.ttl", "go_graph_clean.ttl")


Есть ли в графе RDF.type? True
Очищенный граф сохранен в go_graph_clean.ttl


In [ ]:
from rdflib import Graph, Namespace, RDF, XSD

g = Graph()
g.parse("go_graph_clean.ttl", format="turtle")

ns1 = Namespace("http://example.org/")

query = """
PREFIX ns1: <http://example.org/>

SELECT ?moves ?winRate ?winner
WHERE {
  ?move a ns1:Move ;
        ns1:moves ?moves ;
        ns1:hasEvaluation ?eval .
  ?eval ns1:winRate ?winRate ;
        ns1:winner ?winner .
}
"""
print("Все ходы и их оценка")
for row in g.query(query):
    print(f"Moves: {str(row.moves)}, WinRate: {float(row.winRate)}, Winner: {str(row.winner)}")

query = """
PREFIX ns1: <http://example.org/>

SELECT ?term ?termType ?source ?timestamp
WHERE {
    ?term a ns1:GoTerm ;
          ns1:termType ?termType ;
          ns1:source ?source ;
          ns1:timestamp ?timestamp .
}
"""
print("Все используемые термины")
for row in g.query(query):
    print(f"Term: {row.term}, termType: {str(row.termType)}, source: {str(row.source)}, timestamp: {str(row.timestamp)}")


Все ходы и их оценка
Moves: W[qc];B[qd];W[pc];B[nc];W[oc];B[od];, WinRate: 2.5, Winner: W
Moves: W[rd];B[re];W[rc];B[qf];W[nd];B[mc], WinRate: 3.6, Winner: B
Все используемые термины
Term: http://example.org/GoTerm_mP5CYuj1Qjs_3-3_point, termType: 3-3_point, source: text,audio, timestamp: 1.42-2.19
Term: http://example.org/GoTerm_mP5CYuj1Qjs_black_color, termType: black_color, source: text,audio, timestamp: 11.36-11.72
Term: http://example.org/GoTerm_mP5CYuj1Qjs_hane_term, termType: hane_term, source: text,audio, timestamp: 10.76-11.36
Term: http://example.org/GoTerm_mP5CYuj1Qjs_white_color, termType: white_color, source: text,audio, timestamp: 10.42-10.76
